In [5]:
import sys
from datetime import datetime
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

## Outline
Pet Inventory App
- `App prompts:`
    - Add a new pet
    - Compute and print current weight of all pets in the system
    - Print all pet information for every pet in the system  
    - Print owner information and pet names for all owners with more than one pet
    - Exit the application
- `Owner class attributes:`
    - Owner name (str)
    - Owner address (str)
    Fed as an input to pet class
    
- `Pet class:`
    - Attr:
        - Pet name (str)
        - DOB (date from datetime)
        - Birth weight (float)
        - Owner (Owner class)
    - Method:
        - Compute weight (overwritten by each child class)
- `Mammal class (inherits from Pet):`
    - Attr:
        - Litter Size (int)
        - Has Claw (bool)
    -  Method:
        - Compute weight (Find the number of days since birth. Weight increases by 10% every 60 days for the first 300 days. Weight is constant after that. Assume that the increase occurs on each 60th day only.)
- `Fish class (inherits from Pets):`
    - Attr:
        - Scale condition: 'smooth' or 'rough' (str)
        - lenght (float)
    -  Method:
        - Compute weight (Weight increases by 5% every 80 days for the first 240 days and then stays constant. Assume that the increase occurs on each 80th day only.)
- `Amphibian class (inherits from Pet):`
    - Attr:
        - Number of limbs (int)
        - Is Venemous (bool)
    -  Method:
        - Compute weight (Weight increases by 5% every 120 days for the first 360 days and then increases by 3% every 120 days. Assume that the increase occurs on each 120th day only.)
- `Exceptions:`
    - Date of birth: Valid Date
    - Birth Weight: Positive number
    - Litter Size: Positive Integer
    - hasClaws: User should enter only 'y' or 'n' in either upper or lower case.
    - scaleCondition: User should only enter 's' or 'r' in either upper or lower case
    - length: Positive integer
    - number of limbs: Positive integer
    - isVenomous: User should enter only 'y' or 'n' in either upper or lower case.

## Defining classes:

In [6]:
class Owner:
    def __init__(self, owner_name: str, owner_address: str):
        self.owner_name = owner_name
        self.owner_address = owner_address
    
    def __str__(self):
        return f"   Owner Name: {self.owner_name}\n   Owner Address: {self.owner_address}"

In [7]:
class Pet:
    def __init__(self, pet_name: str, date_of_birth: datetime, birth_weight: float, owner: Owner):
        self.pet_name = pet_name
        self.date_of_birth = date_of_birth
        self.birth_weight = 0.0625 * birth_weight
        self.owner = owner
        
    def compute_weight(self):
        pass
        
    def __str__(self):
        date_out = self.date_of_birth.strftime("%d-%m-%y")
        pet_return = f"\nPet Name: {self.pet_name}"
        pet_return += f"\nBirth Date = {date_out}"
        pet_return += f"\nWeight at birth = {self.birth_weight}"
        pet_return += f"\nOwner details: \n{self.owner}"
        
        return pet_return
    

In [8]:
class Mammal(Pet):
    def __init__(self, pet_name: str, date_of_birth: datetime, birth_weight: float, owner: Owner, litter_size: int, has_claw: str):
        super().__init__(pet_name, date_of_birth, birth_weight, owner)
        self.litter_size = litter_size
        self.has_claw = has_claw
        
    def compute_weight(self):
        days_old = (datetime.now() - self.date_of_birth).days
        growth = 60
        current_weight = self.birth_weight
        while (growth <= 300) and (growth <= days_old):
            current_weight += (0.1 * self.birth_weight)
            growth += 60
        return round(current_weight, 2)
    
    def __str__(self):
        mammal_return = "----------\nPet Type: Mammal"
        mammal_return += super().__str__()
        mammal_return += f"\nLitter Size = {self.litter_size}"
        mammal_return += f"\nHas Claw = {self.has_claw}"
        
        return mammal_return

In [9]:
class Fish(Pet):
    def __init__(self, pet_name: str, date_of_birth: datetime, birth_weight: float, owner: Owner, scale_condition: str, length: float):
        super().__init__(pet_name, date_of_birth, birth_weight, owner)
        self.scale_condition = scale_condition
        self.length = length
        
    def compute_weight(self):
        days_old = (datetime.now() - self.date_of_birth).days
        growth = 80
        current_weight = self.birth_weight
        while (growth <= 240) and (growth <= days_old):
            current_weight += (0.05 * self.birth_weight)
            growth += 80
        return round(current_weight, 2)
        
    def __str__(self):
        fish_return = "----------\nPet Type: Fish"
        fish_return += super().__str__()
        fish_return += f"\nScale Condition = {self.scale_condition}"
        fish_return += f"\nLength = {self.length}"
        
        return fish_return

In [10]:
class Amphibian(Pet):
    def __init__(self, pet_name: str, date_of_birth: datetime, birth_weight: float, owner: Owner, number_of_limbs: int, is_venomous: str):
        super().__init__(pet_name, date_of_birth, birth_weight, owner)
        self.number_of_limbs = number_of_limbs
        self.is_venomous = is_venomous
        
    def compute_weight(self):
        days_old = (datetime.now() - self.date_of_birth).days
        growth = 120
        current_weight = self.birth_weight
        while (growth <= 480) and (growth <= days_old):
            if growth <= 360:
                current_weight += (0.05 * self.birth_weight)
            else:
                current_weight += (0.03 * self.birth_weight)
            growth += 120
        return round(current_weight, 2)
        
    def __str__(self):
        amphibian_return = "----------\nPet Type: Amphibian"
        amphibian_return += super().__str__()
        amphibian_return += f"\nNumber Of Limbs = {self.number_of_limbs}"
        amphibian_return += f"\nIs Venomous = {self.is_venomous}"
        
        return amphibian_return
    

## Defining interface: Pet Tracker Application

In [11]:
def menu_options():
    
    pet_list = read_from_file()
    pet_count_cursor = len(pet_list)
    
    run = True
    while run:
        print("---------------------------------")
        print("Select an option between 1 and 5: ")
        print("1. Add a new pet")
        print("2. Print current weight of all pets in the system")
        print("3. Print all pet information for all pets in the system")
        print("4. Information of all owners with more than one pet")
        print("5. Exit the application")
        
        try:
            option = int(input("Enter your option: "))
            if (option < 1) or (option > 5):
                raise ValueError
                
        except ValueError:
            print("ValueError: Plese select an option between 1 & 5")
            
        else:
            if option == 1:
                print("Option selected: 1")
                pet_list.append(option_one())
                
            elif option == 2:
                print("Option selected: 2")
                option_two(pet_list)
                
            elif option == 3:
                print("Option selected: 3")
                option_three(pet_list)
                
            elif option == 4:
                print("Option selected: 4")
                option_four(pet_list)
                
            else:
                print("Option selected: 5")
                option_five(pet_list, pet_count_cursor)
                run = False

In [12]:
def option_one():
    
    '''
    Option 1: Create Pet objects based on the type entered
    '''
    
    run = True
    while run:
        print("-------------")
        print("Type of pet: ")
        print("1. Mammals")
        print("2. Fish")
        print("3. Amphibians")
        
        try:
            type_of_pet = int(input("Enter value between 1 & 3"))
            if (type_of_pet < 1) or (type_of_pet > 3):
                raise ValueError
                
        except ValueError:
            print("ValueError: Please enter a value between 1 & 3")
            
        else:
            
            pet_name, date_of_birth, birth_weight = pet_input()
            
            if type_of_pet == 1:
                owner_name, owner_address = owner_input()
                owner_inst = Owner(owner_name, owner_address)
                litter_size, has_claw = mammal_input()
                pet_inst = Mammal(pet_name, date_of_birth, birth_weight, owner_inst, litter_size, has_claw)
                
            elif type_of_pet == 2:
                owner_name, owner_address = owner_input()
                owner_inst = Owner(owner_name, owner_address)
                scale_condition, length = fish_input()
                pet_inst = Fish(pet_name, date_of_birth, birth_weight, owner_inst, scale_condition, length)
                
            else:
                owner_name, owner_address = owner_input()
                owner_inst = Owner(owner_name, owner_address)
                number_of_limbs, is_venomous = amphibian_input()
                pet_inst = Amphibian(pet_name, date_of_birth, birth_weight, owner_inst, number_of_limbs, is_venomous)
                
            run = False
            
    return pet_inst

In [13]:
def option_two(pet_list):
    
    '''
    Option 2: Compute and print current weight of all pets in the system
    '''
    
    for obj in pet_list:
        print(f"{obj.pet_name}'s current weight in pounds is {obj.compute_weight()}")

In [14]:
def option_three(pet_list):
    
    '''
    Option 3: Print all pet information for every pet in the system
    '''
            
    mammal_df = pd.DataFrame()
    fish_df = pd.DataFrame()
    amphibian_df = pd.DataFrame()

    for idx, obj in enumerate(pet_list):
        
        temp = obj.__dict__.copy()
        temp.update(temp['owner'].__dict__)
        temp.pop('owner')
        
        if obj.__class__.__name__ == 'Mammal':
            tempm_df = pd.DataFrame(temp, index = [idx])
            mammal_df = pd.concat([mammal_df, tempm_df], axis = 0)
            
        elif obj.__class__.__name__ == 'Fish':
            tempaf_df = pd.DataFrame(temp, index = [idx])
            fish_df = pd.concat([fish_df, tempaf_df], axis = 0)
            
        else:
            tempa_df = pd.DataFrame(temp, index = [idx])
            amphibian_df = pd.concat([amphibian_df, tempa_df], axis = 0)
            
    print("===" * 38)
    print("Data of all Mammals in the database")
    print("===" * 38)
    print(mammal_df.to_markdown(index = False))
    print()
    print("===" * 39)
    print("Data of all Fish in the database")
    print("===" * 39)
    print(fish_df.to_markdown(index = False))
    print()
    print("===" * 40 + "=")
    print("Data of all Amphibians in the database")
    print("===" * 40 + "=")
    print(amphibian_df.to_markdown(index = False))

In [15]:
def option_four(pet_list):
    
    '''
    Option 4: Print owner information and pet names for all owners with more than one pet
    '''
    
    track = defaultdict(list)
    for idx, obj in enumerate(pet_list):
        track[obj.owner.owner_name + obj.owner.owner_address].append(idx)
        
    for uniq in track:
        if len(track[uniq]) > 1:
            print(f"{pet_list[track[uniq][0]].owner.owner_name} lives in {pet_list[track[uniq][0]].owner.owner_address} and has {len(track[uniq])} pets.")
            print("The pet names are:")
            for j in track[uniq]:
                print("\t" + pet_list[j].pet_name)

In [16]:
def option_five(pet_list, cursor):
    print("Saving objects to .dat file")
    save = open("pet_trial.dat", "ab")
    for obj in pet_list[cursor:]:
        pickle.dump(obj, save)
    save.close()
    print("Exiting the application...")
    return 

## Defining all inputs:

In [17]:
def owner_input():
    owner_name = input("Enter owner name: ")
    owner_address = input("Enter owner address: ")
    
    return (owner_name, owner_address)

In [18]:
def pet_input():
    
    pet_name = input("Enter the name of the pet: ")
    
    run = True
    while run:
        try:
            date_of_birth = input("Enter the birth date of the pet in DD-MM-YY format: ")
            date_of_birth = datetime.strptime(date_of_birth, "%d-%m-%y")
        except ValueError:
            print("ValueError: Please enter a valid date format")
        else:
            run = False
    
    run = True
    while run:
        try:
            birth_weight = float(input("Enter the weight of the pet at birth (in ounces): "))
            if birth_weight < 0:
                raise ValueError
        except ValueError:
            print("ValueError: Enter a valid weight: ")
        else:
            run = False
            
    return (pet_name, date_of_birth, birth_weight)

In [19]:
def mammal_input():
    
    run = True
    while run:
        try:
            litter_size = int(input("Enter the litter size of the mammal: "))
            if litter_size < 0:
                raise ValueError
        except ValueError:
            print("ValueError: Enter a positive value for litter size")
        else:
            run = False
    
    run = True
    while run:
        try:
            has_claw = input("Does the mammal have claws? (y/n) ")
            if has_claw.lower() not in ('y','n'):
                raise ValueError
        except ValueError:
            print("ValueError: Please enter y or n")
        else:
            run = False
    
    return (litter_size, has_claw)

In [20]:
def fish_input():

    run = True
    while run:
        try:
            scale_condition = input("Does the fish have scales? (s/r) ")
            if scale_condition.lower() not in ('s','r'):
                raise ValueError
        except ValueError:
            print("ValueError: Please enter s or r")
        else:
            run = False
            
    run = True
    while run:
        try:
            length = int(input("Enter the length of the fish: "))
            if length < 0:
                raise ValueError
        except ValueError:
            print("ValueError: Enter a positive value for fish length")
        else:
            run = False
    
    return (scale_condition, length)

In [21]:
def amphibian_input():
    
    run = True
    while run:
        try:
            number_of_limbs = int(input("Enter the number of libs in the amphibian: "))
            if number_of_limbs < 0:
                raise ValueError
        except ValueError:
            print("ValueError: Enter a positive value for number of limbs")
        else:
            run = False
            
    run = True
    while run:
        try:
            is_venomous = input("Is the amphibian venomous? (y/n) ")
            if is_venomous.lower() not in ('y','n'):
                raise ValueError
        except ValueError:
            print("ValueError: Please enter y or n")
        else:
            run = False
            
    return (number_of_limbs, is_venomous)

In [22]:
def read_from_file():
    
    saved_object = []
    
    try:
        objects = open("pet_trial.dat", "rb")
    except FileNotFoundError:
        print("Object file (.dat) not found!")
        return saved_object
    else:
        
        try:
            while True:
                saved_object.append(pickle.load(objects))
        except EOFError:
            objects.close()
            return saved_object

In [23]:
# def get_df(pet_list):
    
#     try:
#         idx = 0
#         out_df = pd.DataFrame()
#         while True:
#             temp = pet_list[idx].__dict__.copy()
#             temp.update(temp['owner'].__dict__)
#             temp.pop('owner')
#             temp = pd.DataFrame(temp, index = [idx])
#             out_df = pd.concat([out_df, temp], axis = 0)
#             idx += 1
            
#     except IndexError:
#         return out_df

## Adding a pet to the database

In [41]:
menu_options()

---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  1


Option selected: 1
-------------
Type of pet: 
1. Mammals
2. Fish
3. Amphibians


Enter value between 1 & 3 1
Enter the name of the pet:  Yogi
Enter the birth date of the pet in DD-MM-YY format:  05-02-23
Enter the weight of the pet at birth (in ounces):  52
Enter owner name:  Eric
Enter owner address:  890 Main
Enter the litter size of the mammal:  5
Does the mammal have claws? (y/n)  n


---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  5


Option selected: 5
Saving objects to .dat file
Exiting the application...


## Current weight of all pets in database

In [42]:
menu_options()

---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  2


Option selected: 2
Kaazi's current weight in pounds is 3.19
Hiss's current weight in pounds is 5.09
Goldie's current weight in pounds is 1.65
Yogi's current weight in pounds is 4.55
---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  5


Option selected: 5
Saving objects to .dat file
Exiting the application...


## Printing all the pets in the databse

In [43]:
menu_options()

---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  3


Option selected: 3
Data of all Mammals in the database
| pet_name   | date_of_birth       |   birth_weight |   litter_size | has_claw   | owner_name   | owner_address   |
|:-----------|:--------------------|---------------:|--------------:|:-----------|:-------------|:----------------|
| Kaazi      | 2022-12-20 00:00:00 |          2.125 |             3 | y          | Amy          | 123 Main        |
| Yogi       | 2023-02-05 00:00:00 |          3.25  |             5 | n          | Eric         | 890 Main        |

Data of all Fish in the database
| pet_name   | date_of_birth       |   birth_weight | scale_condition   |   length | owner_name   | owner_address   |
|:-----------|:--------------------|---------------:|:------------------|---------:|:-------------|:----------------|
| Goldie     | 2021-10-31 00:00:00 |         1.4375 | r                 |       15 | Brett        | 345 Main        |

Data of all Amphibians in the database
| pet_name   | date_of_birth       |   birth_weight |

Enter your option:  5


Option selected: 5
Saving objects to .dat file
Exiting the application...


## Displaying owners with more than one pet

In [48]:
menu_options()

---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  4


Option selected: 4
Amy lives in 123 Main and has 2 pets.
The pet names are:
	Kaazi
	Hiss
---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  5


Option selected: 5
Saving objects to .dat file
Exiting the application...


## Exiting the application

In [49]:
menu_options()

---------------------------------
Select an option between 1 and 5: 
1. Add a new pet
2. Print current weight of all pets in the system
3. Print all pet information for all pets in the system
4. Information of all owners with more than one pet
5. Exit the application


Enter your option:  5


Option selected: 5
Saving objects to .dat file
Exiting the application...
